In [2]:
!pip install pandasql

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26798 sha256=11a9e83de30a69aff1e9a9fcdcf4d28cd8d27387631a08c80a0da39ece23e542
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\68\5d\a5\edc271b998f909801d7956959f699b976cc9896075dc47c153
Successfully built pandasql


In [3]:
from pandasql import sqldf

# SpaceX Falcon 9 Launch Analysis Using SQL

This notebook performs SQL-based analysis on the cleaned Falcon 9 launch dataset using the `pandasql` library. It helps uncover patterns in launch success rates, payload behavior, booster performance, and more.

In [4]:
import pandas as pd
from pandasql import sqldf

# Load cleaned dataset
df = pd.read_csv(r'C:\Users\user\Downloads\falcon9_cleaned.csv')

# Setup SQL query function
pysqldf = lambda q: sqldf(q, globals())

# Preview data
df.head()

,Date,Booster_Version,Launch_Site,Payload,PayloadMass,Orbit,Customer,Outcome,Landing_Outcome,Class
0,2010-06-04,F9 v1.0 B0003,CCAFS LC-40,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success,Failure (parachute),1
1,2010-12-08,F9 v1.0 B0004,CCAFS LC-40,"Dragon demo flight C1, two CubeSats, barrel of...",0,LEO (ISS),NASA (COTS) NRO,Success,Failure (parachute),1
2,2012-05-22,F9 v1.0 B0005,CCAFS LC-40,Dragon demo flight C2,525,LEO (ISS),NASA (COTS),Success,No attempt,1
3,2012-10-08,F9 v1.0 B0006,CCAFS LC-40,SpaceX CRS-1,500,LEO (ISS),NASA (CRS),Success,No attempt,1
4,2013-03-01,F9 v1.0 B0007,CCAFS LC-40,SpaceX CRS-2,677,LEO (ISS),NASA (CRS),Success,No attempt,1


## 1. Total Successful vs Failed Launches

This query counts how many launches succeeded (`Class = 1`) and how many failed (`Class = 0`).

In [5]:
query1 = """
SELECT Class, COUNT(*) AS Count
FROM df
GROUP BY Class
"""
pysqldf(query1)

,Class,Count
0,0,1
1,1,100


## 2. Launch Sites with the Highest Success Count

This query shows which launch site had the most successful launches.

In [6]:
query2 = """
SELECT Launch_Site, COUNT(*) AS Successes
FROM df
WHERE Class = 1
GROUP BY Launch_Site
ORDER BY Successes DESC
"""
pysqldf(query2)

,Launch_Site,Successes
0,CCAFS SLC-40,34
1,KSC LC-39A,25
2,CCAFS LC-40,25
3,VAFB SLC-4E,16


## 3. Success Rate by Orbit Type

This query calculates the success rate (percentage) for each orbit type.

In [7]:
query3 = """
SELECT Orbit,
       COUNT(*) AS Total,
       SUM(Class) AS Successes,
       ROUND(SUM(Class)*100.0 / COUNT(*), 2) AS Success_Rate_Percent
FROM df
GROUP BY Orbit
ORDER BY Success_Rate_Percent DESC
"""
pysqldf(query3)

,Orbit,Total,Successes,Success_Rate_Percent
0,Sub-orbital,1,1,100.00
1,SSO,6,6,100.00
2,Polar LEO,8,8,100.00
3,MEO,3,3,100.00
4,LEO,25,25,100.00
5,HEO,2,2,100.00
6,GTO,30,30,100.00
7,LEO (ISS),26,25,96.15


## 4. Booster Versions with Most Successes

This query identifies which booster versions had the highest number of successful launches.

In [8]:
query4 = """
SELECT Booster_Version, COUNT(*) AS Successes
FROM df
WHERE Class = 1
GROUP BY Booster_Version
ORDER BY Successes DESC
"""
pysqldf(query4)

,Booster_Version,Successes
0,F9 v1.1,5
1,F9 v1.1 B1017,1
2,F9 v1.1 B1016,1
3,F9 v1.1 B1015,1
4,F9 v1.1 B1014,1
...,...,...
91,F9 B4 B1039.1,1
92,F9 B4 B1043.2,1
93,F9 B4 B1041.2,1
94,F9 B4 B1040.2,1


## 5. Average Payload Mass by Launch Outcome

This query compares the average payload mass for failed and successful launches.

In [9]:
query5 = """
SELECT Class,
       ROUND(AVG(PayloadMass), 2) AS Avg_PayloadMass
FROM df
GROUP BY Class
"""
pysqldf(query5)

,Class,Avg_PayloadMass
0,0,1952.00
1,1,6180.15


## 6. Success Rate Over the Years

This query shows how the success rate has changed year by year.

In [10]:
# Add a Year column first
df['Year'] = pd.to_datetime(df['Date']).dt.year

query6 = """
SELECT Year,
       COUNT(*) AS Total_Launches,
       SUM(Class) AS Successful_Launches,
       ROUND(SUM(Class)*100.0 / COUNT(*), 2) AS Success_Rate
FROM df
GROUP BY Year
ORDER BY Year
"""
pysqldf(query6)

,Year,Total_Launches,Successful_Launches,Success_Rate
0,2010,2,2,100.00
1,2012,2,2,100.00
2,2013,3,3,100.00
3,2014,6,6,100.00
4,2015,7,6,85.71
5,2016,8,8,100.00
6,2017,18,18,100.00
7,2018,20,20,100.00
8,2019,11,11,100.00
9,2020,24,24,100.00


## ✅ Summary of SQL Insights

- Success rate improved in recent years.
- CCAFS and KSC LC-39A sites have the highest number of successful launches.
- GEO and LEO orbits show high success rates.
- Some booster versions like F9 B5 are more successful.